# Egzersiz

## Bir mamogram kitlesinin iyi huylu mu yoksa kötü huylu mu olduğunu tahmin edin

UCI deposundaki "mammographic masses" açık veri kümesini kullanacağız (kaynak: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

Bu veriler, mamogramlarda tespit edilen 961 kitle örneğini içerir ve aşağıdaki özellikleri içerir:


1. BI-RADS değerlendirmesi: 1 ila 5 (sıralı)
2. Yaş: hastanın yıl cinsinden yaşı (tam sayı)
3. Şekil: kütle şekli: yuvarlak=1 oval=2 lobüler=3 düzensiz=4 (nominal)
4. Marj: kütle marjı: sınırlı=1 mikrolobule=2 belirsiz=3 belirsiz=4 sivri uçlu=5 (nominal)
5. Yoğunluk: kütle yoğunluğu yüksek=1 iso=2 düşük=3 yağ içeren=4 (sıralı)
6. Şiddet: iyi huylu=0 veya kötü huylu=1 (binominal)
   


Mamografi sonuçlarından kaynaklanan yanlış pozitiflerden dolayı birçok gereksiz acı ve ameliyat ortaya çıkar. Denetimli makine öğrenimi aracılığıyla bunları yorumlamanın daha iyi bir yolunu bulabilirsek, birçok hayatı iyileştirebilir.

## Senin görevin

Çok Katmanlı bir Algılayıcı oluşturun ve kitleleri özelliklerine göre iyi huylu veya kötü huylu olarak sınıflandırmak için eğitin.

Verilerin temizlenmesi gerekiyor; birçok satır eksik veri içerir ve aykırı değerler olarak tanımlanabilen hatalı veriler de olabilir.

Önce verilerinizi normalleştirmeyi unutmayın! Ve farklı topolojiler, optimize ediciler ve hiper parametrelerle denemeler yapın.

%80'in üzerinde doğruluk üzerine çıkabilir misiniz?

## Başlayalım: verilerinizi hazırlayın

Mammographic_masses.data.txt dosyasını bir Pandas DataFrame aktararak başlayın (ipucu: read_csv kullanın) ve ona bir göz atın.

In [1]:
ROOT_DIR = "https://raw.githubusercontent.com/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART2/Day11-DeepLearning/notebooks/"

DATASET_FILE_PATH = ROOT_DIR + "datasets/mammographic_masses.data.txt"

**Yapılacaklar:**
pandas kutuphanesini import edin ve DATASET_FILE_PATH dosya yolundaki veri kümesini okuyun.
(İpucu: .txt uzantısı olsa da csv olarak okuyabilirsiniz.)

In [2]:
#pandas kutuphanesini import edin ve DATASET_FILE_PATH dosya yolundaki veri kümesini okuyun.
#(İpucu: .txt uzantısı olsa da csv olarak okuyabilirsiniz.)
import pandas as pd

masses_data = pd.read_csv(DATASET_FILE_PATH)
masses_data.head()

,5,67,3,5.1,3.1,1
0,4,43,1,1,?,1
1,5,58,4,5,3,1
2,4,28,1,1,3,0
3,5,74,1,5,?,1
4,4,65,1,?,3,0


**Yapılacaklar:**
Eksik verileri (bir ? ile gösterilir) NaN'ye dönüştürmek ve uygun sütun adlarını (BI_RADS, age, shape, margin, density, ve severity) eklemek için read_csv içindeki isteğe bağlı parametreleri (na_values ve names) kullandığınızdan emin olun:

(Pandas dökümantasyondan yararlanabilirsiniz)

In [3]:
#kolon isimlerini vererek ve eksik verileri NaN'ya dönüştürerek okuyun.
masses_data = pd.read_csv(DATASET_FILE_PATH, na_values=['?'], names = ['BI-RADS', 'age', 'shape', 'margin', 'density', 'severity'])
masses_data.head()

,BI-RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [4]:
masses_data.shape

(961, 6)

**Yapılacaklar:**
Verilerin temizlenmesi gerekip gerekmediğini değerlendirin; NaN kontrolünü tüm kolonlar için yapın

In [5]:
# check for missing values in variables
masses_data.isnull().sum()

BI-RADS      2
age          5
shape       31
margin      48
density     76
severity     0
dtype: int64

Veri setinde epeyce eksik değerler var. Verileri eksik olan her satırı düşürmeden önce, bunu yaparken verilerimizi önyargılı hale getirmediğimizden emin olalım. Ne tür verilerin eksik alanları olduğuyla ilgili herhangi bir korelasyon var gibi görünüyor mu? Olsaydı, geri dönüp bu verileri doldurmaya çalışmamız gerekirdi.

**Yapılacaklar:**
Eksik veriler rastgele dağılmış görünüyorsa, devam edin ve eksik veriler içeren satırları silin. İpucu: dropna() kullanın.

In [6]:
#eksik veriler içeren satırları silin
masses_data.dropna(inplace=True)
masses_data.isnull().sum()

BI-RADS     0
age         0
shape       0
margin      0
density     0
severity    0
dtype: int64

Yapılacaklar:
Eksik verileri içeren satırları sildikten sonra veri kümesinin satır sayısını kontrol edin. 830 adet örnek kalmalı.

In [7]:
#veri kümesi satır satısı kontrolü
len(masses_data)

830

Ardından, Pandas DataFrame'leri scikit_learn tarafından kullanılabilecek numpy dizilerine dönüştürmeniz gerekecek.

**Yapılacaklar:** 
Yalnızca çalışmak istediğimiz öznitelik verilerini (**age, shape, margin, ve density**) ve sınıfları (**severity**) değerlerini **all_features** ve **all_classes** dizi değişkenlerine aktarın. 

Ayrıca bir özniteliklerin kolon isimlerini içeren diziye de ihtiyacımız olacağından, **feature_names** adıyla **kolon isimlerini** de bir diziye tanımlayın.

In [8]:
masses_data['age'].values

array([67., 58., 28., 57., 76., 42., 36., 60., 54., 52., 59., 54., 56.,
       42., 59., 75., 45., 55., 46., 54., 57., 39., 81., 60., 67., 55.,
       78., 50., 62., 64., 67., 74., 80., 49., 52., 60., 57., 74., 49.,
       45., 64., 73., 68., 52., 66., 25., 74., 64., 60., 67., 67., 44.,
       68., 58., 62., 73., 80., 59., 54., 62., 33., 57., 45., 71., 59.,
       56., 57., 55., 84., 51., 24., 66., 33., 59., 40., 67., 75., 86.,
       66., 46., 59., 65., 53., 67., 80., 55., 47., 62., 63., 71., 41.,
       57., 71., 66., 47., 34., 59., 67., 41., 23., 42., 87., 68., 64.,
       54., 86., 21., 53., 44., 54., 63., 45., 71., 49., 49., 66., 19.,
       35., 74., 37., 81., 59., 34., 79., 60., 41., 50., 85., 46., 66.,
       73., 55., 49., 49., 51., 48., 58., 72., 46., 43., 46., 69., 43.,
       76., 46., 59., 57., 45., 43., 45., 57., 79., 54., 40., 63., 52.,
       38., 72., 80., 76., 62., 64., 42., 64., 63., 24., 72., 63., 46.,
       33., 76., 36., 40., 58., 43., 42., 32., 57., 37., 70., 56

In [9]:
#all_features, all_classes ve feature_names'i tanımla
all_features = masses_data[['age', 'shape','margin', 'density']].values
all_classes = masses_data['severity'].values

feature_names = ['age', 'shape', 'margin', 'density']

all_features

array([[67.,  3.,  5.,  3.],
       [58.,  4.,  5.,  3.],
       [28.,  1.,  1.,  3.],
       ...,
       [64.,  4.,  5.,  3.],
       [66.,  4.,  5.,  3.],
       [62.,  3.,  3.,  3.]])

In [10]:
all_features.shape

(830, 4)

In [11]:
all_classes.shape

(830,)

In [12]:
feature_names

['age', 'shape', 'margin', 'density']

**Yapılacaklar:**
Modellerimizden bazıları giriş verilerinin normalleştirilmesini gerektirir, bu nedenle öznitelik verilerini normalleştirin. 

İpucu: preprocessing.StandardScaler() kullanın.

In [13]:
#tüm öznitelikleri standart hale getirin.
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()

all_features_scaled = scaler.fit_transform(all_features)
all_features_scaled

array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [ 0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [ 0.42406719,  0.17563638,  0.11923341,  0.24046607]])

**Yapılacaklar:**
Tüm öznitelikleri %90-%10 şeklinde eğitim ve test olarak ayırın.

In [14]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(all_features_scaled, all_classes, test_size = 0.1, random_state = 0)

In [15]:
X_train.shape

(747, 4)

In [16]:
X_test.shape

(83, 4)

Şimdi Keras'ı kullanarak gerçek bir Çok Katmanlı Algılayıcı (MultiLayer Perceptron) modeli kurun:

**Yapılacaklar: Model Mimarisi**
*  6 nöronlu bir katmana giren 4 öznitelik girişi olan giriş katmanı (Dense Layer). Relu ve ağırlıklar normal dağılım şeklinde başlatılsın. (kernel_initializer='normal')
*   İkili sınıflandırmaya sahip çıktı katmanı (iyi veya kötü huylu) (Dense Layer) Çıkış katmanı olduğudan aktivasyon fonksiyonu sigmoid ve ağırlıklar normal dağılım şeklinde başlatılsın. (kernel_initializer='normal')
*   rmsprop optimizer ve loss binary_crossentropy olacak şekilde metrik olarak da accuracy tanımlayarak modelinizi derleyin.



In [17]:
import tensorflow as tf
#model olusturma

model = tf.keras.Sequential()

#4 feature inputs going into an 6-unit layer (more does not seem to help - in fact you can go down to 4)
model.add(tf.keras.layers.Dense(6, input_dim=4, kernel_initializer='normal', activation='relu'))
# Output layer with a binary classification (benign or malignant)
model.add(tf.keras.layers.Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model; rmsprop seemed to work best





model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

Model özeti aşağıdaki gibi olmalı

```
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
=================================================================
 dense (Dense)               (None, 6)                 30        
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
=================================================================
Total params: 37
Trainable params: 37
Non-trainable params: 0
```

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 30        
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 37
Trainable params: 37
Non-trainable params: 0
_________________________________________________________________


**Yapılacaklar:**
Model eğitimini 20 epoch şeklinde başlatın.
Validation_data olarak test kümesini kullanın.

In [19]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    verbose=2,
                    validation_data=(X_test, y_test))

Epoch 1/20
24/24 - 4s - loss: 0.6867 - accuracy: 0.7831 - val_loss: 0.6811 - val_accuracy: 0.7831 - 4s/epoch - 157ms/step
Epoch 2/20
24/24 - 0s - loss: 0.6773 - accuracy: 0.7992 - val_loss: 0.6703 - val_accuracy: 0.7590 - 81ms/epoch - 3ms/step
Epoch 3/20
24/24 - 0s - loss: 0.6648 - accuracy: 0.8086 - val_loss: 0.6565 - val_accuracy: 0.7711 - 73ms/epoch - 3ms/step
Epoch 4/20
24/24 - 0s - loss: 0.6497 - accuracy: 0.8099 - val_loss: 0.6408 - val_accuracy: 0.7711 - 83ms/epoch - 3ms/step
Epoch 5/20
24/24 - 0s - loss: 0.6329 - accuracy: 0.8086 - val_loss: 0.6229 - val_accuracy: 0.7831 - 80ms/epoch - 3ms/step
Epoch 6/20
24/24 - 0s - loss: 0.6144 - accuracy: 0.8086 - val_loss: 0.6053 - val_accuracy: 0.7831 - 78ms/epoch - 3ms/step
Epoch 7/20
24/24 - 0s - loss: 0.5965 - accuracy: 0.8059 - val_loss: 0.5867 - val_accuracy: 0.7711 - 80ms/epoch - 3ms/step
Epoch 8/20
24/24 - 0s - loss: 0.5770 - accuracy: 0.8059 - val_loss: 0.5664 - val_accuracy: 0.7831 - 75ms/epoch - 3ms/step
Epoch 9/20
24/24 - 0s - 

**Yapılacaklar:**
model başarımınını test kümesi için ölçün: (evaluate)
Test loss ve test acc değerlerini yazdırın.

In [20]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.44864439964294434
Test accuracy: 0.7831325531005859
